# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [1]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [2]:
challenge_id = 'Fuyao' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

# print(query_results)

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

**Tip**: to solve a linear program you can use the following import:

```
from scipy.optimize import linprog
```



In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

model = gp.Model("miqp")

x = model.addVars(n, vtype=GRB.BINARY, name="x")

x_bin = {i: 2 * x[i] - 1 for i in range(n)}

expr = gp.QuadExpr()

for i in range(num_queries):
    linear_expr = gp.LinExpr()
    for j in range(n):
        linear_expr += queries[i][j] * x_bin[j]
    
    expr += (linear_expr - query_results[i]) * (linear_expr - query_results[i])

model.setObjective(expr, GRB.MINIMIZE)

model.setParam('TimeLimit', 30)
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
else:
    print("No optimal solution found.")

solution = [x[i].x for i in range(n)]
x_values = [1 if v > 0.5 else -1 for v in solution]
# print(x_values)


best_query_result = query(challenge_id, np.array(x_values), submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")

Restricted license - for non-production use only - expires 2025-11-24
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 8845H w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 100 columns and 0 nonzeros
Model fingerprint: 0xf0896d10
Model has 4771 quadratic objective terms
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [4e+01, 5e+03]
  QObjective range [3e+01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 94082.000000
Found heuristic solution: objective 66546.000000
Found heuristic solution: objective 33562.000000
Found heuristic solution: objective 33546.000000
Presolve time: 0.13s
Presolved: 0 rows, 100 columns, 0 nonzeros
Pr

In [4]:
# best_query_number = np.argmax(query_results)
# best_query = queries[best_query_number]
# best_query_result = query(challenge_id, best_query, submit=False)
# print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")

In [5]:
# challenge_id
# def test(method, repeat_time=100):
#     res = []
#     for _ in range(repeat_time):
#         queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
#         query_results = query(challenge_id, queries)

#         x = method(queries, query_results)
#         # print(queries.shape, query_results.shape)
#         # print(x)
#         best_query_result = query(challenge_id, x, submit=True)
#         print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")
#         # print((1 + best_query_result / n) / 2)
#         res.append((1 + best_query_result / n) / 2)
#     return np.mean(res)

In [6]:
# # # Method 1
# def linear_regression(queries, query_results):
#     X = queries
#     x = np.linalg.inv(X.T @ X) @ X.T @ query_results
#     x = np.sign(x)
#     return x 
# # test(linear_regression)
# challenge_id = 'Hero123'
# queries = np.random.choice([-1,+1], size=(num_queries,n)) 
# query_results = query(challenge_id, queries)
# x = linear_regression(queries, query_results)

# best_query_result = query(challenge_id, x, submit=True)
# print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")   
